In [ ]:
#%pip install pptx_renderer groq

In [1]:
from pptx_renderer import PPTXRenderer
from groq import Groq

import os
from dotenv import load_dotenv
from types import SimpleNamespace

# Load environment variables from .env
load_dotenv()

# Get Groq API key
api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=api_key)

# model = "llama-3.3-70b-versatile"
# model = "llama3-8b-8192"
model = "gemma2-9b-it"



# Main topic of the presentation
main_topic = "Application of Artificial Intelligence in Education"
# main_topic = "Generative AI"
subtopics_count = 7

# Function to generate content via Groq
def generate_content(prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"""
             You generate clear, concise content for one text element of the slide of the educational presentation.
             the main topic of the presentation is {main_topic}.
             Do not write introductions or explanations — just the content."""},           
            {"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

# Define group topics

prompt = (
    f"List {subtopics_count} subtopics of the theme {main_topic}. "
    "Provide only the list as a Python list of strings."
)

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are an assistant that generates Python lists of strings."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5,
    max_tokens=150
)

group_topics = eval(response.choices[0].message.content.strip())

# group_topics = [
#     "AI for Personalized Learning",
#     "AI in Educational Content Creation",
#     "AI for Student Assessment and Feedback",
#     "AI-Powered Virtual Assistants and Tutoring"
# ]


# Generate content for the title slide
main_title = main_topic
main_subtitle = generate_content(f"Write a short introductory sentence for the topic '{main_topic}' in English.")


groups_data = []

for idx, group_topic in enumerate(group_topics, start=0):
    group_title = group_topic
    group_intro = generate_content(f"Write a brief introduction for the topic '{group_topic}' in 3 short sentences in English.")

    content_title_1 = "Advantages"
    content_text_1 = generate_content(f"List the key advantages of using {group_topic} in education in up to 5 short bullet items in English.")

    content_title_2 = "Practical Examples"
    content_text_2 = generate_content(f"Briefly describe practical examples of applying {group_topic} in education up to 5 short bullet item in English.")

    content_title_3 = "Development Prospects"
    content_text_3 = generate_content(f"Describe the future potential and development prospects of {group_topic} in English in one paragraph.")

    image = f"./img/{idx%3+1}.jpg"

    groups_data.append({
        "group_title": group_title,
        "group_intro": group_intro,
        "content_title_1": content_title_1,
        "content_text_1": content_text_1,
        "content_title_2": content_title_2,
        "content_text_2": content_text_2,
        "content_title_3": content_title_3,
        "content_text_3": content_text_3,
        "image":image
    })

# Convert dictionaries to objects for attribute-style access
groups_data = [SimpleNamespace(**g) for g in groups_data]

# Generate final summary
final_summary = generate_content(f"Write a brief overall conclusion for the entire presentation on the topic '{main_topic}' with subtopics {group_topics} in English in one paragraph.")

# Create presentation with pptx-renderer
renderer = PPTXRenderer("complex_template.pptx")

renderer.render(
    "final_presentation_eng.pptx",
    {
        "main_title": main_title,
        "main_subtitle": main_subtitle,
        "final_title":"SUMMARY",
        "final_summary": final_summary
    },
    loop_groups=[
        {
            "start": 1,
            "end": 4,
            "iterable": groups_data,
            "variable": "group"
        }
    ]
)

print("✅ Presentation created: final_presentation.pptx")


✅ Presentation created: final_presentation.pptx
